# Bottle Data

### Purpose
This notebook provides a quick outline and example of working with OOI Discrete Summary Spreadsheet bottle data. It includes how to load it, how to parse data quality flags, how to derive some important values, etc.

At each [boiler plate stuff about OOI cruises]

### Data
The cruise data is hosted on OOI's Alfresco document server at https://alfresco.oceanobservatories.org, hereafter reffered to as Alfresco. Cruise data is hosted under OOI > {Array} > Cruise Data > {Cruise}. Cruise data includes the following key pieces of information: Cruise Reports, Ship Data (e.g. sonars, underway instruments, etc.), CTD and Water Sampling Data.

We are principally concerned with the Water Sampling data, which can be found under {Cruise} > Ship Data > Water Sampling. The Water Sampling folder contains both digitized and scanned CTD logs as well as the results from the labs for each .

In [1]:
import os, sys, re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [10]:
sys.path.append("..")
from bottle_utils import *

---
### Niskin & Discrete Bottle Sampling
These are discrete water samples collected via Niskin Bottle casts during deployment and recovery of the moored instrumentation. The data is downloaded from OOI Alfresco website as excel files. Parameters sampled are:
* Oxygen
* Salinity
* Nutrients: phosphate, nitrate, nitrite, ammonium, silicate
* Chlorophyll
* Carbon system: Dissolved inorganic carbon, total alkalinity, pH

First, we're going to load, clean, and process the discrete water sampling.

In [6]:
bottle_data = pd.read_excel("../data/bottle_data/Irminger_Sea-02_AT30-01_Discrete_Summary.xlsx")
bottle_data.head()

,Cruise,Station,Target Asset,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast,Cast Flag,Bottom Depth at Start Position [m],CTD File,...,Discrete pH Replicate Flag,Calculated Alkalinity [umol/kg],Calculated DIC [umol/kg],Calculated pCO2 [uatm],Calculated pH,Calculated CO2aq [umol/kg],Calculated Bicarb [umol/kg],Calculated CO3 [umol/kg],Calculated Omega-C,Calculated Omega-A
0,AT30-01,1,release test,42.547833,-62.584,2015-08-07T12:39:53.000Z,1,*0000000000000100,1723,C:\data\ctd\at30_001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
1,AT30-01,1,release test,42.547833,-62.584,2015-08-07T12:39:53.000Z,1,*0000000000000100,1723,C:\data\ctd\at30_001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
2,AT30-01,1,release test,42.547833,-62.584,2015-08-07T12:39:53.000Z,1,*0000000000000100,1723,C:\data\ctd\at30_001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
3,AT30-01,1,release test,42.547833,-62.584,2015-08-07T12:39:53.000Z,1,*0000000000000100,1723,C:\data\ctd\at30_001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
4,AT30-01,1,release test,42.547833,-62.584,2015-08-07T12:39:53.000Z,1,*0000000000000100,1723,C:\data\ctd\at30_001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999


Replace the fill value of ```-9999999``` with NaNs. Some columns were imported as floats and others as strings or objects, so this needs to be done twice:

In [7]:
bottle_data = bottle_data.replace(to_replace="-9999999", value=np.nan)
bottle_data = bottle_data.replace(to_replace=-9999999, value=np.nan)

Convert times from strings to pandas datetime objects and remove the timezone:

In [11]:
bottle_data["Start Time [UTC]"] = bottle_data["Start Time [UTC]"].apply(lambda x: convert_times(x))
bottle_data["CTD Bottle Closure Time [UTC]"] = bottle_data["CTD Bottle Closure Time [UTC]"].apply(lambda x: convert_times(x))

Some of the data returned from the labs which process samples contain non-numeric indicators for if data were below detection threshold. These entries typically have a "less than" sign. Convert any values with a "<", which indicates a value not statistically significant from zero, with zero:

In [12]:
bottle_data = bottle_data.applymap(not_statistically_sigificant)

#### Filter data
Next, we want to filter the data based on the data flag associated with the sample. First, we should drop all of the ```NaN``` values in the Niskin column to eliminate entries without any bottle samples. While these CTD casts may be of interest, the bottle data does not have any data of which we can make use of.

In [14]:
bottle_data = bottle_data.dropna(subset=["Niskin/Bottle Position"])

#### Interpret Flags
Next, we want to parse the Data Quality Flags in the bottle data. 

We intrept the data into the standard QARTOD flaging scheme of:
* 1 = good
* 2 = not run
* 3 = suspect
* 4 = bad
* 9 = missing

The "Replicate Flags" are reduced into a boolean value indicating that either there is a replicate sample or not. By replicate, we mean if there are more than one sample collected from a depth, e.g. two Niskins were closed at a particular depth and oxygen sampled from both. It does *not* mean that a particular analysis was repeated on the same sample multiple times, e.g. repeating nutrient analysis on a single Niskin to derive a final value.

In [15]:
for col in bottle_data.columns:
    if "Flag" in col:
        if "CTD" in col and "File" not in col:
            bottle_data[col] = bottle_data[col].apply(lambda x: interp_ctd_flag(x))
        elif "Discrete" in col:
            bottle_data[col] = bottle_data[col].apply(lambda x: interp_discrete_flag(x))
        elif "Replicate" in col:
            bottle_data[col] = bottle_data[col].apply(lambda x: interp_replicate_flag(x))
        elif "Niskin" in col:
            bottle_data[col] = bottle_data[col].apply(lambda x: interp_niskin_flag(x))
        else:
            pass

---
### Calculate Physical Properties
With the bottle samples cleaned up and the flags parsed into something simpler and easier to read, we next want to calculate the conservative seawater properties, such as density, conservative temperature, absolute salinity, etc., using the updated Thermodynamic Equations of State - 2010 (TEOS-10) algorithms. These are implemented with the ```gsw``` package. 

This step will allow us to correct for pressure and temperature effect between lab analysis and in-situ measurements, as well as allow for better comparison between depths and locations in the same array. 